# Leaflet Finder


credit to https://www.cs.rice.edu/~vs3/comp422/lecture-notes/comp422-lec24-s08-v2.pdf  page 38


- partition the adjacency matrix across processors
- run indepentent connected components on each processor (find_partial_connected_components)
- merge spanning forests until one remains  (unify_components)

For Merging:
I use intersection and union functions:
- intersection: I check if there is at least one common node, which means MSTs are connected
- union: if there is a common node, I do union and add to the set of connected componnents, otherwise I add each node alone.

I run the merging function pairwise, and repeat till only one list remain.

In [1]:
filename = 'atom_pos_132K.npy'

In [75]:
import numpy as np
coord_matrix = np.load(filename)
coord_matrix = coord_matrix[:800]   # TODO: remove 
matrix_size = coord_matrix.shape[0] 
arranged_coord = list()

In [76]:
arranged_coord = list()
part_size = 5
for i in range(1,matrix_size+1,part_size):
    for j in range(i,matrix_size,part_size):
        # The arranged_elem contains a tuple with the data needed to calculate
        # in a window. The first part of the tuple is a list that contains
        # two numpy arrays. The second element has indices of the first element
        # of both arrays.
        arranged_elem = ([coord_matrix[i-1:i-1+part_size],coord_matrix[j-1:j-1+part_size]],[i,j])
        arranged_coord.append(arranged_elem)

In [77]:
dist_Matrix = sc.parallelize(arranged_coord,len(arranged_coord))

In [78]:
connected_components = dist_Matrix.flatMap(find_partial_connected_components).reduce(merge_spanning_trees)

[array([52]),
 array([111]),
 array([176]),
 array([211]),
 array([498]),
 array([628]),
 array([642, 643]),
 array([647]),
 array([672]),
 array([673]),
 array([675]),
 array([687]),
 array([698]),
 array([705]),
 array([712, 714, 715]),
 array([713]),
 array([716, 717]),
 array([718]),
 array([721, 722]),
 array([723]),
 array([724]),
 array([726, 727, 728, 729]),
 array([731, 732, 733, 734]),
 array([746, 749]),
 array([747, 748]),
 array([756]),
 array([757, 759]),
 array([758]),
 array([761, 762, 763]),
 array([766, 767, 768, 769, 770]),
 array([771]),
 array([773]),
 array([776, 777]),
 array([781, 782, 783]),
 array([786]),
 array([791, 792, 794, 795]),
 array([793]),
 array([796, 797, 799, 800]),
 array([798])]

## Merge spanning forests pairwise until only one remains

In [55]:
def merge_spanning_trees(c1,c2):
    while len(c2)!=0:
        temp = c2[0]
        c2.pop(0)
        for i,item in enumerate(c1):
            if item.intersection(temp):
                temp.union(item)
                c1.pop(i)
        c1.append(temp)
    
    return c1

## Run Independent Conected components 

In [6]:
def find_partial_connected_components(data,cutoff=15.00):
    
    window = data[0]
    i_index = data[1][0]
    j_index = data[1][1]
    
    import networkx as nx
    graph = nx.Graph()

    ## pairwise distances
    for i in range(0,len(window[0])):
        for j in range(0,len(window[1])):
            if np.sqrt(sum((window[0][i] - window[1][j]) ** 2)) <= cutoff:
                graph.add_edge(i+i_index,j+j_index)    # fix indexes
                
    # partial connected components
    connected_components = nx.connected_components(graph)
                        
    for x in connected_components:
        yield [x]


In [80]:
## Test union find
a = [4,3,6,9,2,8,5,7,6,1,6]
b = [3,8,5,4,1,9,0,2,1,0,0]

uf = QuickUnionUF(len(a))

for i in xrange(len(a)):
    p = a[i]
    q = b[i]
    print 'ids: '
    print uf.ids
    
    if not uf.connected(p,q):
        uf.union(p,q)
        print "%d + %d "  % (p,q)
        
        
## Sucess!!!!!

        
        
        
    


ids: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
4 + 3 
ids: 
[0, 1, 2, 4, 4, 5, 6, 7, 8, 9, 10]
3 + 8 
ids: 
[0, 1, 2, 4, 4, 5, 6, 7, 4, 9, 10]
6 + 5 
ids: 
[0, 1, 2, 4, 4, 6, 6, 7, 4, 9, 10]
9 + 4 
ids: 
[0, 1, 2, 4, 4, 6, 6, 7, 4, 4, 10]
2 + 1 
ids: 
[0, 2, 2, 4, 4, 6, 6, 7, 4, 4, 10]
ids: 
[0, 2, 2, 4, 4, 6, 6, 7, 4, 4, 10]
5 + 0 
ids: 
[6, 2, 2, 4, 4, 6, 6, 7, 4, 4, 10]
7 + 2 
ids: 
[6, 2, 2, 4, 4, 6, 6, 2, 4, 4, 10]
6 + 1 
ids: 
[6, 2, 6, 4, 4, 6, 6, 2, 4, 4, 10]
ids: 
[6, 6, 6, 4, 4, 6, 6, 2, 4, 4, 10]


In [79]:
class QuickUnionUF:
    ids = []  # ints
    sz = []   # I guess
    
    def __init__(self,N):
        for i in xrange(N):
            QuickUnionUF.ids.append(i)
            QuickUnionUF.sz.append(1)  # i guess
            
    
    def root(self,i):
        while (i!=self.ids[i]):
            self.ids[i] = self.ids[self.ids[i]]
            i =  self.ids[i]
        return i  
        
    def connected(self,p,q):
        return self.root(p) == self.root(q)
    
    def union(self,p,q):
        pid = self.ids[p]
        qid = self.ids[q]
        
        if pid==qid:
            return
        
        if self.sz[pid] < self.sz[qid]:
            self.ids[pid] = qid
            self.sz[qid] += self.sz[pid]
        else:
            self.ids[qid] = pid
            self.sz[pid] += self.sz[qid]

In [32]:
 a = [{1,2,3,4,5},{7,8,9,45},{34,21,66}]

In [33]:
a

[{1, 2, 3, 4, 5}, {7, 8, 9, 45}, {21, 34, 66}]

In [35]:
del a[1]

In [37]:
a.append({7, 8, 9, 45})

In [69]:
a.append(b)

In [68]:
b = {3,4,5}

[{1, 2, 3, 4, 5}, {21, 34, 66}, {7, 8, 9, 45}, {3, 4, 5}]

In [73]:
aa = [{1, 2, 3, 5},
 {4},
 {1, 2, 3, 5, 6, 7, 8, 9},
 {1, 2, 3, 5, 12},
 {4, 13},
 {1, 2, 3, 5, 19, 20},
 {2, 3, 21, 22},]

In [81]:
x = sc.parallelize({1, 2, 3, 5,6,7,8})
y = sc.parallelize({4,5})

In [82]:
x.union(y).collect()

[1, 2, 3, 5, 4, 5]

In [84]:
sc.parallelize({1, 2, 3, 5,6,7,8},2)


ParallelCollectionRDD[68] at parallelize at PythonRDD.scala:475

In [89]:
c1 = {1, 2, 3, 5}
c2 =  {4,5}

In [90]:
c1.intersection(c2)

{5}

In [10]:
aa = [[{1, 2, 3, 5}],  [{4}],  [{1, 2, 3, 5, 6, 7, 8, 9}],  [{1, 2, 3, 5, 12}], [{4, 13}], [{1, 2, 3, 5, 19, 20}],
 [{2, 3, 21, 22}], [{4, 23, 24, 25}], [{4, 26, 27}], [{1, 5, 33, 34, 35}]]

In [45]:
def test (c1,c2):
    while len(c2)!=0:
        temp = c2[0]
        c2.pop(0)
        for i,item in enumerate(c1):
            if item.intersection(temp):
                temp.union(item)
                c1.pop(i)
        c1.append(temp)
    return c1


In [46]:
test(aa[0],[{1, 2, 3, 5},{4,5,6,7}])

set([1, 2, 3, 5])


In [25]:
c2 = [{1}]

In [27]:
c2.pop(0)

{1}

In [51]:
c2 = [{1, 2, 3, 5},{4,5,6,7}]

In [53]:
for i,item in enumerate(c2):
    print item

set([1, 2, 3, 5])
set([4, 5, 6, 7])


In [49]:
c2

[{4, 5, 6, 7}]

set([1])
